## Loading in Dataset

https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/ufo-scrubbed-geocoded-time-standardized-00.csv

In [3]:
import pandas as pd
import numpy as np
import altair as alt

# Because this dataset was not showing column names, I had to create/add them.

colm_names = ["date_time", "city", "state", "country", "object_shape","duration_seconds",
              "duration","comments","occurance_date", "latitude", "longitude"]
df = pd.read_csv(
    "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/ufo-scrubbed-geocoded-time-standardized-00.csv",
    names = colm_names)

# Then I decided to convert the time so that it wourked better with what I was trying to do

df["date_time"] = pd.to_datetime(df["date_time"], errors = "coerce")
df["year"] = df["date_time"].dt.year
df["month"] = df["date_time"].dt.to_period("M").dt.to_timestamp()

df = df.dropna(subset=["date_time", "latitude", "longitude"])

df

HTTPError: HTTP Error 404: Not Found

In [ ]:
top_10_states = ( df[(df['country'] == 'us') & df['state'].notna()].groupby("state")
                 .size() .reset_index(name = "count")
                 .sort_values("count", ascending = False)
                 .head(10) )

chart_a = (
    alt.Chart(top_10_states)
    .mark_bar()
    .encode(
        x = alt.X("count:Q", title = "Sightings", axis = alt.Axis(titleFontSize = 15, labelFontSize = 14)
        ),
        y = alt.Y("state:N", sort = "-x", title = "State", axis = alt.Axis(titleFontSize = 15, labelFontSize = 14)),
        tooltip = ["state:N", "count:Q"]
    )
    .properties(
        title = alt.TitleParams("Top 10 States with U.F.O Sightings", fontSize = 20),
        width = 1000,
        height = 400
    )
)

chart_a


In [ ]:
alt.data_transformers.disable_max_rows()

df["object_shape"] = df["object_shape"].fillna("unknown")

# Interactivity
brush = alt.selection_interval(encodings = ["x"])

# line graph (top)
chart_b = (
    alt.Chart(df_small)
      .mark_line()
      .encode(
          x = alt.X("yearmonth(date_time):T" , title = "Month/Years"),
          y = alt.Y("count():Q", title = "Sightings per Month")
      )
      .properties(width = 650, height = 250, title = "U.F.O Sightings Over Time")
      .add_params(brush)
)

# Histograph (bottom)
chart_c = (
    alt.Chart(df_small)
      .transform_filter(brush)
      .mark_bar()
      .encode(
          x = alt.X("count():Q", title = "Sightings"),
          y = alt.Y("object_shape:N", sort = "-x", title = "Object Shape"),
          color = alt.Color("object_shape:N", legend = None, scale = alt.Scale(scheme = "rainbow")),
          tooltip = ["object_shape:N", "count():Q"]
      )
      .properties(width = 650, height = 300, title = "U.F.O Shapes Over the Years")
)

final_charts = chart_b & chart_c
final_charts


In [ ]:
chart_a.save("chart_by_states.json")
final_charts.save("chart_b_and c_interactive.json")